# HW7: Comparing MNB & SVM with Kaggle Sentiment Data

## OVERVIEW

---
### VECTORIZERS USED:
    CountVectorizer
    TfidfVectorizer

### MODELS USED:
    Multinomial Naive Bayes (MNB)
    Support Vector Machines (SVM)
---

---
#### VECTORIZATION PARAMS:
    Binary
    Stopwords
    Unigrams, Bigrams
    Min & Max df
---

#### TODO:
    Stemming?
    Vadar + TextBlob

### FUNCTION & PACKAGE PARTY

In [4]:
## =======================================================
## TOKENIZING
## =======================================================
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

## =======================================================
## VECTORIZING
## =======================================================
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## ----- VECTORIZORS
unigram_bool_cv = CountVectorizer(encoding='latin-1', binary=True, min_df=5, stop_words='english')
unigram_bool_cv_v2 = CountVectorizer(encoding='latin-1', binary=True, min_df=5, stop_words='english', token_pattern=r'(?u)\b[a-zA-Z]{2,}\b' )

unigram_cv = CountVectorizer(encoding='latin-1', binary=False, min_df=5, stop_words='english')
bigram_cv = CountVectorizer(encoding='latin-1', ngram_range=(1,2), min_df=5, stop_words='english')
unigram_tv = TfidfVectorizer(encoding='latin-1', use_idf=True, min_df=5, stop_words='english')
bigram_tv = TfidfVectorizer(encoding='latin-1', use_idf=True, ngram_range=(1,2), min_df=5, stop_words='english')

## =======================================================
## MODELING
## =======================================================
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

## ----- CLASSIFIERS
mnb = MultinomialNB()
svm = LinearSVC(C=1)

def get_test_train_vec(X,y,vectorizer):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    return X_train_vec, X_test_vec, y_train, y_test

def run_classifier(X_train_vec, X_test_vec, y_train, y_test, labels, target_names, classifier):
    clf = classifier
    clf.fit(X_train_vec,y_train)
    print(clf.score(X_test_vec,y_test))
    return clf
    
def get_model(X, y, labels, target_names, classifier, vec):
    X_train_vec, X_test_vec, y_train, y_test = get_test_train_vec(X,y,vec)
    model = run_classifier(X_train_vec, X_test_vec, y_train, y_test, labels, target_names, classifier)
    return model
    
## =======================================================
## VISUALIZING
## =======================================================
from tabulate import tabulate
import pandas as pd

def return_features(vec, model):
    for i,feature_probability in enumerate(model.coef_):
        print('============ Sentiment Score: ', i)
        df1 = pd.DataFrame(sorted(zip(feature_probability, vec.get_feature_names()))[:10])
        df2 = pd.DataFrame(sorted(zip(feature_probability, vec.get_feature_names()))[-10:])
        df3 = pd.concat([df1, df2], axis=1)
        print(tabulate(df3, tablefmt="fancy_grid", headers=["Most","Likely","Least","Likely"], floatfmt=".2f"))
        

### DATA GOES HERE:

In [5]:
# import pandas as pd
train=pd.read_csv("kaggle-sentiment/train.tsv", delimiter='\t')
y=train['Sentiment'].values
X=train['Phrase'].values

# TASK 1

## MNB | Vectorizer 1


In [6]:
vec = unigram_bool_cv
classifier = mnb

model = get_model(X,y,[0,1,2,3,4],['0','1','2','3','4'], classifier, vec)
return_features(vec, model)

0.606401384083045
============ Sentiment Score:  0
╒════╤════════╤══════════╤═════════╤════════════╕
│    │   Most │ Likely   │   Least │ Likely     │
╞════╪════════╪══════════╪═════════╪════════════╡
│  0 │ -10.48 │ 102      │   -5.95 │ time       │
├────┼────────┼──────────┼─────────┼────────────┤
│  1 │ -10.48 │ 10th     │   -5.94 │ minutes    │
├────┼────────┼──────────┼─────────┼────────────┤
│  2 │ -10.48 │ 127      │   -5.93 │ characters │
├────┼────────┼──────────┼─────────┼────────────┤
│  3 │ -10.48 │ 13th     │   -5.93 │ story      │
├────┼────────┼──────────┼─────────┼────────────┤
│  4 │ -10.48 │ 14       │   -5.90 │ comedy     │
├────┼────────┼──────────┼─────────┼────────────┤
│  5 │ -10.48 │ 16       │   -5.70 │ just       │
├────┼────────┼──────────┼─────────┼────────────┤
│  6 │ -10.48 │ 163      │   -5.20 │ like       │
├────┼────────┼──────────┼─────────┼────────────┤
│  7 │ -10.48 │ 168      │   -5.07 │ bad        │
├────┼────────┼──────────┼─────────┼────────────┤

## SVM | Vectorizer 1

In [156]:
vec = unigram_bool_cv
classifier = svm

model = get_model(X,y,[0,1,2,3,4],['0','1','2','3','4'], classifier, vec)
return_features(vec, model)

0.6245033961296937
============ Sentiment Score:  0
╒════╤════════╤═══════════╤═════════╤════════════════╕
│    │   Most │ Likely    │   Least │ Likely         │
╞════╪════════╪═══════════╪═════════╪════════════════╡
│  0 │  -1.81 │ hawke     │    1.63 │ cesspool       │
├────┼────────┼───────────┼─────────┼────────────────┤
│  1 │  -1.71 │ collar    │    1.66 │ pompous        │
├────┼────────┼───────────┼─────────┼────────────────┤
│  2 │  -1.69 │ giddy     │    1.69 │ stinks         │
├────┼────────┼───────────┼─────────┼────────────────┤
│  3 │  -1.59 │ swimfan   │    1.70 │ distasteful    │
├────┼────────┼───────────┼─────────┼────────────────┤
│  4 │  -1.57 │ blue      │    1.71 │ unwatchable    │
├────┼────────┼───────────┼─────────┼────────────────┤
│  5 │  -1.45 │ dogtown   │    1.72 │ disappointment │
├────┼────────┼───────────┼─────────┼────────────────┤
│  6 │  -1.41 │ victim    │    1.76 │ unbearable     │
├────┼────────┼───────────┼─────────┼────────────────┤
│  7 │  -1.41